In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle
import sys
sys.path.append("../../../Benchmark/")
from benchmarker import Benchmarker

In [ ]:
import os
os.chdir("../../../datatset/MultiEmbryo2/")

Prepare data

In [3]:
# rna = sc.read_h5ad("rna_annot.h5ad")
# rna.obs_names = [i.split("-")[0] for i in rna.obs_names]
# rna.obs_names
# # sc.pp.filter_genes(rna, min_counts=3)
# rna.write("rna_annot2.h5ad")

In [27]:
rna = sc.read_h5ad("rna_annot2.h5ad")
rna

AnnData object with n_obs × n_vars = 1949 × 32285
    obs: 'annot'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'annot_colors'
    obsm: 'spatial'

In [5]:
rna.var.drop_duplicates()

,gene_ids,feature_types,genome
Xkr4,ENSMUSG00000051951,Gene Expression,mm10
Gm1992,ENSMUSG00000089699,Gene Expression,mm10
Gm19938,ENSMUSG00000102331,Gene Expression,mm10
Gm37381,ENSMUSG00000102343,Gene Expression,mm10
Rp1,ENSMUSG00000025900,Gene Expression,mm10
...,...,...,...
AC124606.1,ENSMUSG00000095523,Gene Expression,mm10
AC133095.2,ENSMUSG00000095475,Gene Expression,mm10
AC133095.1,ENSMUSG00000094855,Gene Expression,mm10
AC234645.1,ENSMUSG00000095019,Gene Expression,mm10


In [6]:
files = ["rna_annot.h5ad", "gs.h5ad"]
adatas = []
for i in files:
    adata = sc.read_h5ad(i)
    adatas.append(adata)

In [7]:
adatas

[AnnData object with n_obs × n_vars = 1949 × 32285
     obs: 'annot'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'annot_colors'
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 1949 × 55291
     obs: 'tsse', 'n_fragment', 'frac_dup', 'frac_mito'
     uns: 'reference_sequences'
     obsm: 'insertion']

In [4]:
# adatas[1].obs_names = [i.split("-")[0] for i in adatas[1].obs_names]

In [5]:
# adatas[1] = adatas[1][adatas[0].obs_names]
# adatas[0].obs["cell_type"] = list(adatas[0].obs["annot"])
# adatas[1].obs["cell_type"] = list(adatas[0].obs["cell_type"])
# adatas[1].obsm["spatial"] = adatas[0].obsm["spatial"].copy()

In [6]:
# import numpy as np
# for i, ad in enumerate(adatas):
#     ad.obs_names = [j+f"_{i}" for j in ad.obs_names]
#     ad.var_names_make_unique()
# adatas[0].obs["batch"] = "RNA"
# adatas[1].obs["batch"] = "ATAC"

In [7]:
# concat = sc.concat(adatas)
# concat.obs["spatial1"] =  list(concat.obsm["spatial"][:,0])
# concat.obs["spatial2"] =  list(concat.obsm["spatial"][:,1])

In [8]:
# concat.write("concat.h5ad")

In [5]:
bm = Benchmarker(R_conda_env="Rbase")

In [10]:
# bm.h5ad2rds(in_file=concat, out_file="concat.rds", verbose=True)

Run all method

In [ ]:
# bm.run(RDS_file_path="concat.rds", H5AD_file_path="concat.h5ad",
#        save_path="../../../result/MultiEmbryo2/embed/", 
#        n_cluster=7, verbose=True, workers=4)

Evaluate

In [6]:
adata = sc.read_h5ad("concat.h5ad")

In [ ]:
res_dict = bm.read_result("../../result/MultiEmbryo2/embed/", 
                          index=list(adata.obs_names),
                          reindex=False,
                          save="../../result/MultiEmbryo2/embed_dict.pkl",
                          methods=bm.all_methods+["BindSC", "GLUE", "Monae", "SCALEX", "scConfluence", "SIMBA"])

In [ ]:
metrics = bm.cal_metrics(adata=adata, batch_key="batch", label_key="cell_type",
                         res_dict=res_dict, methods="all", verbose=True, rep=1,
                         min_max_scale=False, save="../../result/MultiEmbryo2/metrics.pkl")

In [21]:
with open("../../result/MultiEmbryo2/metrics.pkl", "rb") as f:
    metrics = pickle.load(f)

In [ ]:
bm.set_plot_params(params_dict={"figure.dpi": 300}, font_file_path="./Helvetica.ttf")

2025-12-21 17:35:09 - INFO - Custom font 'Helvetica' has been set


In [ ]:
save_dir = "../../..//figures/Embryo2/"

In [17]:
# bm.plot_heatmap(metric_df=metrics[0],
#                 save=f"{save_dir}/summary_heatmap.pdf")

In [10]:
from benchmarker import split_adata, transform_coord
import numpy as np
spatial = [ad.obsm["spatial"] for ad in split_adata(adata)]
spatial = transform_coord(spatial, vertical=True, margin_size=0.05, axis="y")

In [19]:
# spatial = [adata.obsm["spatial"]]
# spatial = transform_coord(spatial, vertical=True,)

In [11]:
bg_dict = {i:"#D4B483" if i in bm.spatial_methods else "#5873a4" for i in res_dict["UMAP"].keys() }
bg_dict["STADIA"] = "#D4B483"

In [21]:
# bm.plot_legend(category_lst=adata.obs["cell_type"], marker="o", ncol=2, save=f"{save_dir}/annot_legend.pdf",
#                handletextpad=0, borderpad=0,
#                labelspacing=0.6, columnspacing=0,)

In [33]:
# bm.plot_legend(category_lst=adata.obs["cell_type"], marker="o", ncol=3, save=f"{save_dir}/annot_legend2.pdf",
#                handletextpad=0, borderpad=0,
#                labelspacing=0.6, columnspacing=0,)

In [28]:
# bm.plot_legend(category_lst=list(range(7)), marker="o", ncol=3, save=f"{save_dir}/cluster_legend.pdf",)
#             #    handletextpad=0, borderpad=0,
#             #    labelspacing=0.6, columnspacing=0,)

In [22]:
# bm.plot_legend(category_lst=list(adata.obs["batch"]), marker="o", ncol=1,
#             #    order=["Visium", "Stereo-seq", "Slide-seq V2"],
#                save=f"{save_dir}/batch_legend.pdf")

In [23]:
# bm.plot_spatial(spatial=spatial, label_dict={"annot": np.array(adata.obs["cell_type"]).reshape(-1,1)},
#                 figsize=(2, 4), frameon=True, inner_gs_row=2, inner_gs_col=1, size=15, ncol=1,
#                 xlabel=["Annotation"], ylabel=None, only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 outer_row_hspace=0.15, outer_col_wspace=0.1,
#                 background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#                 xlabel_pad=0.015,
#                 save=f"{save_dir}/annot_spatial_plot.pdf")

In [13]:
from matplotlib import rcParams
if len(rcParams["axes.prop_cycle"].by_key()["color"]) >= 10:
        cc = rcParams["axes.prop_cycle"]()
        palette = [next(cc)["color"] for _ in range(10)]
cmap = {str(i): palette[i] for i in range(10)}

In [15]:
# bm.plot_spatial(spatial=spatial, label_dict=res_dict["Cluster"],
#                 figsize=(18, 14), frameon=True, inner_gs_row=2, inner_gs_col=1, size=20, ncol=8,
#                 xlabel="name", ylabel=["RNA", "ATAC"], only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 # order=["Graspot", "SLAT", "STAligner", "DeepST", "scVI",  "Seurat", "LIGER",],
#                 outer_row_hspace=0.15, outer_col_wspace=0.05, inner_common_camp=False,
#                 background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#                 xlabel_pad=0.015, palette=cmap, save_dpi=600,
#                 save=f"{save_dir}/all_spatial.pdf")

In [78]:
# bm.plot_spatial(spatial=spatial, label_dict=res_dict["Cluster"],
#                 figsize=(10, 4.0), frameon=True, inner_gs_row=2, inner_gs_col=1, size=18, ncol=5,
#                 xlabel="name", ylabel=["RNA", "ATAC"], only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 order=["SLAT", "scVI", "GLUE", "Monae", "BindSC"],
#                 inner_row_hspace=0.18, outer_col_wspace=0.05, inner_common_camp=True,
#                 background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#                 xlabel_pad=0.012, palette=cmap, save_dpi=600,
#                 save=f"{save_dir}/unpaired_spatial.pdf")

In [18]:
# bm.plot_umap(embed_dict=res_dict["UMAP"], batch_dict=res_dict["Batch"], annot_list=list(adata.obs["cell_type"]), 
#              figsize=(14, 12), frameon=True, inner_gs_row=2, inner_gs_col=1, size=3, ncol=8,
#              ylabel=["Batch", "Cell type"], only_show_top=False,  xlabel="name", only_show_left=True,
#              background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#             #  order=["Unintegrated", "SLAT", "Seurat", "GLUE", "SCALEX", "BindSC"],
#              axis_width=1.2, axis_color="lightgrey",
#              outer_col_wspace=0.05, save_dpi=600, outer_row_hspace=0.15,
#              ylabel_pad=0.018, xlabel_pad=0.015, merge=True, merge_margin_size=0.4,
#              save=f"{save_dir}/all_umap.pdf")

In [28]:
paired_methods = ["SpatialGlue", "COSMOS", "MISO", "PRESENT"]
embed, cluster = {}, {}
for i in paired_methods:
    embed[i] = pd.read_csv(f"../../result/MultiEmbryo2/embed/embed_{i}.csv", index_col=0)
    cluster[i] = pd.read_csv(f"../../result/MultiEmbryo2/embed/cluster_{i}.csv", index_col=0)
    if i == "PRESENT":
        # pass
        cluster[i].index = [i.split("-")[0] for i in cluster[i].index]
        embed[i].index = cluster[i].index.copy()
        embed[i] = embed[i].reindex(rna.obs_names)
        cluster[i] = cluster[i].reindex(rna.obs_names)
    else:
        embed[i].index = rna.obs_names.copy()
    cluster[i] = np.array(cluster[i].iloc[:,0])

In [57]:
spatial = [split_adata(adata)[0].obsm["spatial"]]
spatial = transform_coord(spatial, vertical=True, margin_size=0.05, axis="y")

In [79]:
# bm.plot_spatial(spatial=spatial,
#                 label_dict=cluster,
#                 figsize=(4.3, 4.5), frameon=True, inner_gs_row=1, inner_gs_col=1, size=18, ncol=2,
#                 xlabel="name",
#                 only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 order=["SpatialGlue", "MISO", "COSMOS", "PRESENT"],
#                 outer_row_hspace=0.2, outer_col_wspace=0.05, inner_common_camp=True,
#                 background_color = lambda x: "#D4B483",
#                 xlabel_pad=0.012,
#                 palette=cmap,
#                 save_dpi=600,
#                 save=f"{save_dir}/paired_spatial.pdf"
#                 )

In [82]:
# bm.plot_umap(embed_dict=res_dict["UMAP"], batch_dict=res_dict["Batch"], annot_list=list(adata.obs["cell_type"]), 
#              figsize=(10.5, 3.9), frameon=True, inner_gs_row=2, inner_gs_col=1, size=3, ncol=6,
#              ylabel=["Batch", "Cell type"], only_show_top=False,  xlabel="name", only_show_left=True,
#              background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#              order=["Unintegrated", "SLAT", "scVI", "GLUE", "Monae", "BindSC"],
#              axis_width=1.2, axis_color="lightgrey",
#              outer_col_wspace=0.05, save_dpi=600,
#              ylabel_pad=0.018, xlabel_pad=0.015, merge=True, merge_margin_size=0.4,
#              save=f"{save_dir}/unpaired_umap.pdf")

In [88]:
# umap = {}
# for i in embed.keys():
#     t_adata = sc.AnnData(X=embed[i])
#     sc.pp.neighbors(t_adata, use_rep="X")
#     sc.tl.umap(t_adata)
#     umap[i] = t_adata.obsm["X_umap"]

In [89]:
# bg_dict = {i:"#D4B483" if i not in bm.spatial_methods else "#5873a4" for i in embed.keys() }

In [87]:
# bm.plot_umap(embed_dict=umap, annot_list=list(rna.obs["annot"]), 
#              figsize=(3.8, 4.1), frameon=True, inner_gs_row=1, inner_gs_col=1, size=5, ncol=2,
#              ylabel=["Cell type"], only_show_top=False,  xlabel="name", only_show_left=True,
#              background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#             order=["SpatialGlue", "MISO", "COSMOS", "PRESENT"],
#              axis_width=1.2, axis_color="lightgrey",
#              outer_col_wspace=0.05, save_dpi=600,  outer_row_hspace=0.2,
#              ylabel_pad=0.018, xlabel_pad=0.012, merge=False, merge_margin_size=0.4,
#              save=f"{save_dir}/paired_umap.pdf")

In [22]:
batch_metric = metrics[0].copy()
batch_metric = batch_metric.loc[:,["Silhouette batch","iLISI", "KBET", "Graph connectivity", "PCR comparison", "Batch correction"]]
batch_metric.loc["Metric Type"] = [i.replace(" ", "\n") for i in batch_metric.loc["Metric Type"]]

In [24]:
# bm.plot_heatmap(metric_df=batch_metric, save=f"{save_dir}/summary_batch_heatmap_all.pdf", total_name="Batch correction", )#insert_marker_row=8, show_top=7, show_bottom=5)

In [29]:
paired_res_dict = {}
paired_res_dict["Cluster"] = cluster.copy()
paired_res_dict["Embed"] = embed.copy()

In [30]:
rna.obs["batch"] = ["b1" if i<1200  else "b2" for i in range(rna.shape[0])] 

In [31]:
bio_metric = metrics[0].copy()
bio_metric = bio_metric.loc[:,['CHAOS', 'PAS', 'Isolated labels', 'NMI', 'ARI', 'Silhouette label', 'cLISI', 'Domain continuity', 'Bio conservation']]
# bio_metric.loc["Metric Type"] = [i.replace(" ", "\n") for i in bio_metric.loc["Metric Type"]]

In [ ]:
paired_metric = bm.cal_metrics(adata=rna, batch_key="batch", label_key="annot",
               res_dict=paired_res_dict, methods="all", verbose=True, rep=1,
               min_max_scale=False, save="../../result/MultiEmbryo2/paired_metrics.pkl")

In [33]:
bio_paired_metirc = paired_metric[0].loc[:,['CHAOS', 'PAS', 'Isolated labels', 'NMI', 'ARI', 'Silhouette label',
       'cLISI', 'Domain continuity', 'Bio conservation']]
bio_paired_metirc = bio_paired_metirc.drop("Metric Type")
bio_metric_all = pd.concat((bio_metric, bio_paired_metirc))#.sort_values("Bio conservation", ascending=False)

In [34]:
bio_metric_all.index = ["*"+i if i in paired_methods else i for i in bio_metric_all.index]


In [39]:
# bm.plot_heatmap(metric_df=bio_metric_all, save=f"{save_dir}/summary_bio_heatmap.pdf", total_name="Bio conservation", insert_marker_row=8, show_top=7, show_bottom=5)